<a href="https://colab.research.google.com/github/sridhartroy/AIML/blob/main/Big_Data_A1_Sri.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Step 1: Install required libraries
!pip install pyspark==3.5.6 spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.4/317.4 MB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.6-py2.py3-none-any.whl size=317895798 sha256=20681fbd4a95f291eb3e65a26e7be1104e26f91fd434105f347427bccb380c34
  Stored in directory: /root/.cache/pip/wheels/64/62/f3/ec15656ea4ada0523cae62a1827fe7beb55d3c8c87174aad4a
Successfully built pyspark
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.5.1
    Uninstalling pyspark-3.5.1:
      Successfully uninstalled pyspark-3.5.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 118.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [1]:
# Step 2: Import libraries
import spacy
from pyspark.sql import SparkSession

# Step 3: Initialize Spark session
spark = SparkSession.builder \
    .appName("NER-WordCount") \
    .getOrCreate()

sc = spark.sparkContext

# Step 4: Download a large text file from Gutenberg
# Example: "Pride and Prejudice" by Jane Austen
!wget -O pride_prejudice.txt https://www.gutenberg.org/files/1342/1342-0.txt

# Step 5: Load text into Spark RDD
rdd = sc.textFile("pride_prejudice.txt")

# Step 6: Load SpaCy model
nlp = spacy.load("en_core_web_sm")

# Step 7: Define function to extract Named Entities
def extract_entities(text):
    doc = nlp(text)
    return [ent.text for ent in doc.ents]

# Step 8: Apply map-reduce (WordCount on Named Entities)
entities_rdd = rdd.flatMap(extract_entities) \
                  .map(lambda ent: (ent, 1)) \
                  .reduceByKey(lambda a, b: a + b) \
                  .sortBy(lambda x: x[1], ascending=False)

# Step 9: Collect & display results
top_entities = entities_rdd.take(20)   # top 20 entities
for ent, count in top_entities:
    print(f"{ent} -> {count}")


--2025-09-23 02:46:35--  https://www.gutenberg.org/files/1342/1342-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 752583 (735K) [text/plain]
Saving to: ‘pride_prejudice.txt’

pride_prejudice.txt 100%[===================>] 734.94K  2.50MB/s    in 0.3s    

2025-09-23 02:46:36 (2.50 MB/s) - ‘pride_prejudice.txt’ saved [752583/752583]

Elizabeth -> 633
Darcy -> 364
Jane -> 294
Bingley -> 243
Bennet -> 232
Wickham -> 190
Collins -> 189
Lydia -> 163
first -> 151
two -> 123
one -> 113
Lizzy -> 95
Gardiner -> 94
Lady Catherine -> 86
Charlotte -> 77
Netherfield -> 71
Kitty -> 57
Meryton -> 56
Longbourn -> 55
London -> 55
